In [1]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download -d gunavenkatdoddi/eye-diseases-classification

100% 733M/736M [00:07<00:00, 140MB/s]
100% 736M/736M [00:07<00:00, 99.0MB/s]


In [12]:
pip install split-folders

In [13]:
import splitfolders

In [17]:
import zipfile
import os

# Path to the zip file
zip_file_path = '/content/eye-diseases-classification.zip'

# Directory where you want to extract the contents
extracted_dir = '/content/extracted_data'

# Create the output directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents into the output directory
    zip_ref.extractall(extracted_dir)

# Now the contents of the zip file are extracted to the 'extracted_data' directory


In [18]:
splitfolders.ratio('/content/extracted_data/dataset', output="output", seed=1337, ratio=(.8, 0.2))

Copying files: 4217 files [00:02, 1424.18 files/s]


In [19]:
IMAGE_SIZE = [224, 224]

In [20]:
import splitfolders
from tensorflow.keras.preprocessing. image import ImageDataGenerator
from PIL import ImageFile
ImageFile. LOAD_TRUNCATED_IMAGES = True
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


In [21]:
from tensorflow.keras.preprocessing. image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2, zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [22]:
training_set = train_datagen.flow_from_directory(
'/content/output/train',
target_size = (224, 224),
batch_size = 64,
class_mode = 'categorical')
test_set = test_datagen. flow_from_directory('/content/output/val',
target_size = (224, 224),
batch_size = 64,
class_mode = 'categorical')

Found 3372 images belonging to 4 classes.
Found 845 images belonging to 4 classes.


In [23]:
VGG19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet',include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


In [24]:
for layer in VGG19.layers:
  layer.trainable = False

In [25]:
x = Flatten()(VGG19.output)

In [28]:
x = Flatten()(VGG19.output)

In [29]:
prediction = Dense(4, activation='softmax')(x)

In [30]:
model = Model(inputs=VGG19.input, outputs=prediction)

In [31]:
model. summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
r = model.fit(
training_set,
validation_data=test_set,
epochs=2,
steps_per_epoch=len(training_set),
validation_steps=len(test_set))

Epoch 1/2
53/53 [==============================] - 3311s 63s/step - loss: 0.5524 - accuracy: 0.7841 - val_loss: 0.6099 - val_accuracy: 0.7479
Epoch 2/2
53/53 [==============================] - 3354s 63s/step - loss: 0.5062 - accuracy: 0.8060 - val_loss: 0.4787 - val_accuracy: 0.8308


In [36]:
model.save('animal.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model("/content/animal.h5")


In [38]:
img = image. load_img(r"/content/output/val/normal/2365_left.jpg",target_size= (224,224))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
preds=model.predict(x)
pred=np.argmax(preds,axis=1)
index=['cataract', 'diabetic_retinopathy', 'glaucoma' , 'normal']
result=str(index[pred[0]])
result

1/1 [==============================] - 1s 1s/step


'normal'